In [57]:
import pandas as pd
import numpy as np


import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation

file = "C:/Users/TYS/Documents/study/Master/INSY662/group project/Real Code/INSY662-Group-Project/Data/dataset_demand.csv"
data = pd.read_csv(file)
data

,date,start_station_cluster,rideable_type,trip_count
0,2022/1/1,0,classic_bike,94
1,2022/1/1,0,electric_bike,196
2,2022/1/1,1,classic_bike,11
3,2022/1/1,1,electric_bike,8
4,2022/1/1,2,classic_bike,37
...,...,...,...,...
12299,2022/12/31,17,electric_bike,189
12300,2022/12/31,18,classic_bike,13
12301,2022/12/31,18,electric_bike,46
12302,2022/12/31,19,classic_bike,9


In [58]:
# Filter data for start_station_cluster 0
#cluster_data = data[data['start_station_cluster'] == 0]

# Separate data for classic bikes and electric bikes
# use data instead of cluster_data
classic_bike_data = data[data['rideable_type'] == 'classic_bike']
electric_bike_data = data[data['rideable_type'] == 'electric_bike']
all_bike_data = data

classic_bike_data.head(), electric_bike_data.head()

(       date  start_station_cluster rideable_type  trip_count
 0  2022/1/1                      0  classic_bike          94
 2  2022/1/1                      1  classic_bike          11
 4  2022/1/1                      2  classic_bike          37
 5  2022/1/1                      3  classic_bike         118
 7  2022/1/1                      4  classic_bike          65,
        date  start_station_cluster  rideable_type  trip_count
 1  2022/1/1                      0  electric_bike         196
 3  2022/1/1                      1  electric_bike           8
 6  2022/1/1                      3  electric_bike         262
 8  2022/1/1                      4  electric_bike          97
 9  2022/1/1                      6  electric_bike           8)

In [59]:
# Convert 'date' to datetime and set as index
classic_bike_data['date'] = pd.to_datetime(classic_bike_data['date'])
electric_bike_data['date'] = pd.to_datetime(electric_bike_data['date'])
all_bike_data['date'] = pd.to_datetime(all_bike_data['date'])
classic_bike_data.set_index('date', inplace=True)
electric_bike_data.set_index('date', inplace=True)
all_bike_data.set_index('date', inplace=True)

# Display the first few rows of each dataset
classic_bike_data.head(), electric_bike_data.head() 

C:\Users\TYS\AppData\Local\Temp\ipykernel_59812\3828104725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classic_bike_data['date'] = pd.to_datetime(classic_bike_data['date'])
C:\Users\TYS\AppData\Local\Temp\ipykernel_59812\3828104725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electric_bike_data['date'] = pd.to_datetime(electric_bike_data['date'])


(            start_station_cluster rideable_type  trip_count
 date                                                       
 2022-01-01                      0  classic_bike          94
 2022-01-01                      1  classic_bike          11
 2022-01-01                      2  classic_bike          37
 2022-01-01                      3  classic_bike         118
 2022-01-01                      4  classic_bike          65,
             start_station_cluster  rideable_type  trip_count
 date                                                        
 2022-01-01                      0  electric_bike         196
 2022-01-01                      1  electric_bike           8
 2022-01-01                      3  electric_bike         262
 2022-01-01                      4  electric_bike          97
 2022-01-01                      6  electric_bike           8)

In [60]:
# Make sure the data is sorted by date
classic_bike_data.sort_index(inplace=True)
electric_bike_data.sort_index(inplace=True)
all_bike_data.sort_index(inplace=True)

# Determine the split point for the classic bike data
split_point_classic = len(classic_bike_data) // 2

# Split the classic bike data into train and test
classic_bike_train = classic_bike_data.iloc[:split_point_classic]
classic_bike_test = classic_bike_data.iloc[split_point_classic:]

# Determine the split point for the electric bike data
split_point_electric = len(electric_bike_data) // 2

# Split the electric bike data into train and test
electric_bike_train = electric_bike_data.iloc[:split_point_electric]
electric_bike_test = electric_bike_data.iloc[split_point_electric:]

split_point_all = len(all_bike_data) // 2

all_bike_train = all_bike_data.iloc[:split_point_all]
all_bike_test = all_bike_data.iloc[split_point_all:]

# You now have the train and test sets for both classic and electric bikes

# Display the first few rows of each dataset
classic_bike_train.head(), electric_bike_train.head(),all_bike_train.head()


(            start_station_cluster rideable_type  trip_count
 date                                                       
 2022-01-01                      0  classic_bike          94
 2022-01-01                      1  classic_bike          11
 2022-01-01                      2  classic_bike          37
 2022-01-01                      3  classic_bike         118
 2022-01-01                      4  classic_bike          65,
             start_station_cluster  rideable_type  trip_count
 date                                                        
 2022-01-01                      0  electric_bike         196
 2022-01-01                      1  electric_bike           8
 2022-01-01                      3  electric_bike         262
 2022-01-01                      4  electric_bike          97
 2022-01-01                      6  electric_bike           8,
             start_station_cluster  rideable_type  trip_count
 date                                                        
 2022-01-01  

In [61]:
from pmdarima import auto_arima
import pandas as pd
from pandas import Timedelta, Timestamp
from statsmodels.tsa.arima.model import ARIMA

# Assuming 'data' is your DataFrame containing all clusters and bike types
clusters = data['start_station_cluster'].unique()
arima_params = {}

for cluster in clusters:
    for bike_type in ['classic_bike', 'electric_bike']:
        # Define the key for storing in the dictionary
        key = (cluster, bike_type)

        # Filter data for the current cluster and bike type
        cluster_bike_data = all_bike_train[(all_bike_train['start_station_cluster'] == cluster) &
                                 (all_bike_train['rideable_type'] == bike_type)]['trip_count'].dropna()

        # Convert the series to float
        cluster_bike_data = cluster_bike_data.astype(float)

        # Convert the index to weekly period
        cluster_bike_data.index = pd.DatetimeIndex(cluster_bike_data.index).to_period('D')

        try:
            # Find the best ARIMA parameters
            model = auto_arima(cluster_bike_data,
                               seasonal=True,
                               trace=True,
                               error_action='warn',
                               suppress_warnings=False,
                               stepwise=True,
                               max_p=3, max_q=3, max_d=2,
                               max_order=None,
                               enforce_stationarity=True,
                               enforce_invertibility=True)

            # Store parameters
            arima_params[key] = {
                'order': model.order,
                'seasonal_order': model.seasonal_order,
                'aic': model.aic()
            }

            # Concise output
            print(f"Cluster {cluster} ({bike_type.capitalize()}): ARIMA{model.order}, Seasonal Order={model.seasonal_order}, AIC={model.aic()}")

        except Exception as e:
            print(f"Failed to find ARIMA model for {bike_type} in cluster {cluster}: {e}")

c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Performing stepwise search to minimize aic


c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.65 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2118.268, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2101.853, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2081.247, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2116.303, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2064.247, Time=0.22 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.20 sec


c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


...

 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.45 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=748.283, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=711.262, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.17 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=746.283, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=703.199, Time=0.05 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=676.981, Time=0.06 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=657.914, Time=0.15 sec


c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=658.382, Time=0.21 sec
 ARIMA(3,1,1)(0,0,0)[0]             : AIC=656.349, Time=0.10 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=659.040, Time=0.09 sec


c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


 ARIMA(3,1,0)(0,0,0)[0]             : AIC=674.991, Time=0.04 sec
 ARIMA(3,1,2)(0,0,0)[0]             : AIC=656.953, Time=0.13 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=701.201, Time=0.03 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=660.068, Time=0.15 sec

Best model:  ARIMA(3,1,1)(0,0,0)[0]          
Total fit time: 1.953 seconds
Cluster 5 (Classic_bike): ARIMA(3, 1, 1), Seasonal Order=(0, 0, 0, 0), AIC=656.3494623345653
Failed to find ARIMA model for electric_bike in cluster 5: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.


In [62]:
c_temp_0 = electric_bike_train[electric_bike_train['start_station_cluster'] == 0]
c_temp_0

,start_station_cluster,rideable_type,trip_count
date,,,
2022-01-01,0,electric_bike,196
2022-01-02,0,electric_bike,194
2022-01-03,0,electric_bike,227
2022-01-04,0,electric_bike,221
2022-01-05,0,electric_bike,282
...,...,...,...
2022-06-28,0,electric_bike,783
2022-06-29,0,electric_bike,721
2022-06-30,0,electric_bike,630


In [63]:
model_temp = ARIMA(c_temp_0['trip_count'], order=(0,1,1))
results_temp = model_temp.fit()
#forecast_temp = results_temp.get_prediction(start='2022-07-02', end='2022-12-31', dynamic=False)
forecast_temp = results_temp.predict(start='2022-01-02', end='2022-01-10', dynamic=False)
forecast_temp

c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


2022-01-02    193.874757
2022-01-03    193.937845
2022-01-04    205.384619
2022-01-05    209.619056
2022-01-06    226.194203
2022-01-07    235.699868
2022-01-08    238.736335
2022-01-09    242.982600
2022-01-10    246.497883
Freq: D, Name: predicted_mean, dtype: float64

In [64]:
classic_start = len(classic_bike_train)
classic_end = len(classic_bike_data) + len(classic_bike_test) - 1
electric_start = len(electric_bike_train)
electric_end = len(electric_bike_data) + len(electric_bike_test) - 1


In [65]:
# def demand_forecasting(cluster_to_predict, bike_type_to_predict, date_to_predict):
#     # Ensure the date is in the proper format
#     date_to_predict = pd.to_datetime(date_to_predict)

#     # Select the appropriate training dataset
#     if bike_type_to_predict == "electric_bike":
#         cluster_temp = electric_bike_train[electric_bike_train['start_station_cluster'] == cluster_to_predict]
#     elif bike_type_to_predict == "classic_bike":
#         cluster_temp = classic_bike_train[classic_bike_train['start_station_cluster'] == cluster_to_predict]
#     else:
#         raise ValueError("Bike type must be 'electric_bike' or 'classic_bike'.")

#     # Ensure there is enough data to fit the model
#     if len(cluster_temp) == 0:
#         raise ValueError(f"No data available for cluster {cluster_to_predict} and bike type {bike_type_to_predict}.")

#     # Fit the ARIMA model
#     model_order = arima_params[(cluster_to_predict, bike_type_to_predict)]['order']
#     model = ARIMA(cluster_temp['trip_count'], order=model_order)
#     results = model.fit()

#     # Forecast up to the date of interest
#     # Assuming the index of the cluster_temp is a DatetimeIndex and is in ascending order
#     forecast_end = (date_to_predict - cluster_temp.index[0]).days
#     forecast = results.get_prediction(start=0, end=forecast_end)

#     # Extract the forecast for the specific date
#     try:
#         predicted_count = forecast.predicted_mean.loc[date_to_predict]
#     except KeyError:
#         raise KeyError(f"No prediction available for {date_to_predict}. Model forecast may not cover this date.")

#     return predicted_count


In [66]:
arima_params

{(0, 'classic_bike'): {'order': (1, 1, 1),
  'seasonal_order': (0, 0, 0, 0),
  'aic': 2064.2471916172635},
 (0, 'electric_bike'): {'order': (3, 1, 3),
  'seasonal_order': (0, 0, 0, 0),
  'aic': 2149.9358103533123},
 (1, 'classic_bike'): {'order': (0, 1, 1),
  'seasonal_order': (0, 0, 0, 0),
  'aic': 1247.1831601606},
 (1, 'electric_bike'): {'order': (0, 0, 0),
  'seasonal_order': (0, 0, 0, 0),
  'aic': 1015.6493122039806},
 (2, 'classic_bike'): {'order': (1, 1, 1),
  'seasonal_order': (0, 0, 0, 0),
  'aic': 1485.6953532642056},
 (3, 'classic_bike'): {'order': (0, 1, 2),
  'seasonal_order': (0, 0, 0, 0),
  'aic': 1896.1942806312684},
 (3, 'electric_bike'): {'order': (3, 1, 3),
  'seasonal_order': (0, 0, 0, 0),
  'aic': 2107.0814507455666},
 (4, 'classic_bike'): {'order': (2, 1, 2),
  'seasonal_order': (0, 0, 0, 0),
  'aic': 1730.1205571208243},
 (4, 'electric_bike'): {'order': (0, 1, 0),
  'seasonal_order': (0, 0, 0, 0),
  'aic': 1884.47066643778},
 (6, 'classic_bike'): {'order': (0, 1,

In [67]:
def demand_forecasting(cluster_to_predict, bike_type_to_predict, date_to_predict):
    # Convert the prediction date to pandas Timestamp for consistency
    date_to_predict = pd.to_datetime(date_to_predict)

    # Select the appropriate training dataset
    if bike_type_to_predict == "electric_bike":
        #cluster_temp = electric_bike_train[electric_bike_train['start_station_cluster'] == cluster_to_predict]
        cluster_temp = electric_bike_data[electric_bike_data['start_station_cluster'] == cluster_to_predict]
    elif bike_type_to_predict == "classic_bike":
        cluster_temp = classic_bike_data[classic_bike_data['start_station_cluster'] == cluster_to_predict]
    else:
        raise ValueError("Bike type must be 'electric_bike' or 'classic_bike'.")
#------------------------------------------------------------------------------------------
    # Filter data for the current cluster and bike type
    #cluster_temp = all_bike_train[(all_bike_train['start_station_cluster'] == cluster_to_predict) &
    #                            (all_bike_train['rideable_type'] == bike_type_to_predict)]['trip_count'].dropna()
    #print(cluster_temp)
    # Convert the series to float
    #cluster_temp = cluster_bike_data.astype(float)

    # Convert the index to weekly period
    #cluster_temp.index = pd.to_datetime(cluster_temp.index).to_period('D')

    #cluster_temp = cluster_temp.asfreq('D')
#-------------------------------------------------------------------------------------------
    # Ensure there is enough data to fit the model
    if len(cluster_temp) < 2:  # You need at least two data points to forecast
        raise ValueError(f"Not enough data to fit ARIMA model for cluster {cluster_to_predict} and bike type {bike_type_to_predict}.")

    # Fit the ARIMA model
    model_order = arima_params[(cluster_to_predict, bike_type_to_predict)]['order']
    model = ARIMA(cluster_temp['trip_count'], order=model_order, freq='D')
    results = model.fit()

    # Forecasting
    forecast_start = len(cluster_temp)  # Start forecasting after the last observed data point
    print(cluster_temp.index[-1])
    forecast_end = forecast_start + (date_to_predict - cluster_temp.index[-1]).days  # Days to forecast
    print(forecast_end)
    dynamic = forecast_end > forecast_start  # Use dynamic forecast if forecasting more than one step
    #forecast = results.get_prediction(start=forecast_start, end=forecast_end, dynamic=dynamic)
    forecast = results.get_prediction(start="2022-01-01", end="2022-12-31", dynamic=dynamic)

    #print(forecast.predicted_mean)

    # Extract the forecast for the specific date
    try:
        predicted_count = forecast.predicted_mean.loc[date_to_predict]
        #predicted_count = forecast.predicted_mean[date_to_predict]
    except KeyError:
        raise KeyError(f"No prediction available for {date_to_predict}. Model forecast may not cover this date.")

    return predicted_count



In [68]:
# Example parameters
cluster_to_predict = 1  # This is the cluster number you want to predict for
bike_type_to_predict = 'classic_bike'  # This can be 'classic_bike' or 'electric_bike'
date_to_predict = '2022-09-30'  # The date you want to make the prediction for, in 'YYYY-MM-DD' format

# Call the demand_forecasting function with the example parameters
predicted_trip_count = demand_forecasting(cluster_to_predict, bike_type_to_predict, date_to_predict)

# Print out the result
#print(f"The predicted trip count for cluster {cluster_to_predict}, bike type '{bike_type_to_predict}', on {date_to_predict} is: {predicted_trip_count}")
# Print the values in three separate rows
print(f"The predicted trip count for cluster {cluster_to_predict}:")
print(f"Bike type '{bike_type_to_predict}':")
print(f"On {date_to_predict} is: {predicted_trip_count}")


NameError: name 'classci_bike_data' is not defined

In [ ]:
row = data.iloc[8649]
print(row)

start_station_cluster                6
rideable_type            electric_bike
trip_count                          21
Name: 2022-09-14 00:00:00, dtype: object


In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# def plot_demand_forecasting(cluster_to_predict, bike_type_to_predict, date_to_predict):
#     # Combine training and test datasets for plotting
#     if bike_type_to_predict == "electric_bike":
#         train_temp = electric_bike_train[electric_bike_train['start_station_cluster'] == cluster_to_predict]
#         test_temp = electric_bike_test[electric_bike_test['start_station_cluster'] == cluster_to_predict]
#     elif bike_type_to_predict == "classic_bike":
#         train_temp = classic_bike_train[classic_bike_train['start_station_cluster'] == cluster_to_predict]
#         test_temp = classic_bike_test[classic_bike_test['start_station_cluster'] == cluster_to_predict]
#     else:
#         raise ValueError("Bike type must be 'electric_bike' or 'classic_bike'.")

#     # Create a date range from the first day of the training data to the last day of the test data
#     #start_date = len(electric_bike_train) #train_temp.index.min()
#     #end_date = len(electric_bike_train) + len(electric_bike_test) -1 #test_temp.index.max()
#     start_date = train_temp.index.min().date()
#     end_date = test_temp.index.max().date()
   
#     if start_date < end_date:
#         print(type(start_date))
#         print(end_date)
#         print()

#     #date_range = pd.date_range(start=start_date, end=end_date, freq='D')
#     date_range = pd.date_range(start=start_date, end=end_date, freq='D')

#     # Initialize a list to store forecasted values
#     forecasted_values = []

#     # Calculate forecasted trip count for each day using demand_forecasting function
#     for current_date in date_range:
#         forecasted_trip_count = demand_forecasting(cluster_to_predict, bike_type_to_predict, date_to_predict)
#         forecasted_values.append(forecasted_trip_count)

#     # Convert the list of forecasted values to a pandas Series
#     forecasted_series = pd.Series(forecasted_values, index=date_range)

#     # Plot the actual and forecasted trip counts
#     plt.figure(figsize=(14, 7))
#     plt.plot(train_temp.index, train_temp['trip_count'], label='Training Data', color='blue')
#     plt.plot(test_temp.index, test_temp['trip_count'], label='Actual Test Data', color='orange')
#     plt.plot(date_range, forecasted_series, label='Forecasted Data', color='green', linestyle='--')
#     plt.title(f'Actual vs Forecasted Trip Counts for {bike_type_to_predict.capitalize()} in Cluster {cluster_to_predict}')
#     plt.xlabel('Date')
#     plt.ylabel('Trip Count')
#     plt.legend()
#     plt.show()

# To use the function, you will need to ensure that:
# - The demand_forecasting function is defined in your environment.
# - The electric_bike_train, electric_bike_test, classic_bike_train, and classic_bike_test datasets are properly set up.
# - The arima_params dictionary is correctly defined with the ARIMA order parameters for each combination of cluster and bike type.


In [ ]:
# Function to forecast demand
def demand_forecasting_with_trend(cluster_to_predict, bike_type_to_predict):
    # Select the appropriate training dataset
    if bike_type_to_predict == "electric_bike":
        cluster_temp = electric_bike_data[electric_bike_data['start_station_cluster'] == cluster_to_predict]
        test_temp = electric_bike_test[electric_bike_test['start_station_cluster'] == cluster_to_predict]
    elif bike_type_to_predict == "classic_bike":
        cluster_temp = classic_bike_data[classic_bike_data['start_station_cluster'] == cluster_to_predict]
        test_temp = classic_bike_test[classic_bike_test['start_station_cluster'] == cluster_to_predict]
    else:
        raise ValueError("Bike type must be 'electric_bike' or 'classic_bike'.")

    # Fit the ARIMA model
    model_order = arima_params[(cluster_to_predict, bike_type_to_predict)]['order']
    model = ARIMA(cluster_temp['trip_count'], order=model_order)
    results = model.fit()

    # Predict for the entire year of 2022
    forecast_end = '2022-12-31'
    forecast = results.get_prediction(start='2022-01-01', end=forecast_end, dynamic=False)
    forecasted_values = forecast.predicted_mean

    # Plotting the training data, forecasted data
    plt.figure(figsize=(14, 7))
    plt.plot(cluster_temp['trip_count'], label='Training Data')
    plt.plot(test_temp['trip_count'],label='Test Data',color='green')
    plt.plot(pd.date_range(start='2022-01-01', end=forecast_end, freq='D'), forecasted_values, 
             label='Forecasted Data', linestyle='--')
    plt.title(f"Actual vs Forecasted Trip Counts for {bike_type_to_predict.replace('_', ' ').title()} in Cluster {cluster_to_predict}")
    plt.legend()
    plt.show()


In [ ]:
# Call the function to plot for cluster 0, electric bike
demand_forecasting_with_trend(1, 'electric_bike')

c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


KeyError: 'The `end` argument could not be matched to a location related to the index of the data.'